In [1]:
from glob import glob
import numpy as np
import pandas as pd
from scipy import stats

from analysis_scripts import second_harm_tools
from analysis_scripts.analyzs_many import get_sea_urchin

/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
sea_urchins = glob("../data/sea_urchin_darkfield/*.h5")


In [3]:
len(sea_urchins)

4

In [4]:
defaults =  {"t_range": slice(None, None)}
params = {}


results = []
spectrums = []
for exp in sea_urchins:
    param = params.get(exp, defaults)
    if param.get("skip"):
        continue
    a = get_sea_urchin(exp, rel_s=0.9, **param)
    if a.last_valid_index() < 0.1:
        continue
    res_sea, spectrum = second_harm_tools.analyse_shape(a, NFFT=125, pca=True, smoothing=58)
    res_sea.index.name = "time"
    res_sea.reset_index(inplace=True)
    res_sea["exp"] = exp
    results.append(res_sea)
    spectrums.append(spectrum)
    
    
res_seas = pd.concat(results)


4


/Data/iff_th2/rode/sperm/experiments/compare_species/nb/analysis_scripts/second_harm_tools.py:73: RuntimeWarning: divide by zero encountered in double_scalars
  taub = 1/fr_m
/Data/iff_th2/rode/sperm/experiments/compare_species/nb/analysis_scripts/second_harm_tools.py:125: RuntimeWarning: divide by zero encountered in true_divide
  mode='full')/c0f,


6
4
5


In [5]:
5*1./res_seas.dropna().omega.mean()*500

57.99345377531046

In [6]:
res_seas["phi_vel_normed_s_c0"] = res_seas.phi_vel_normed_s/(res_seas.C0**2)
res_seas["c2_plus_cm"] = 0.04*res_seas.C2_SIN + 0.36*res_seas.kappa_mean_s

In [7]:
res = []
for spectrum in spectrums:
    res.append((spectrum[::100,:,0].flatten(), np.sqrt(spectrum[::100,:,1].flatten())))
xx = np.hstack(list(zip(*res))[0])
yy = np.hstack(list(zip(*res))[1])
    

In [8]:
power, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="mean")
power_std, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="std")

freq = (bin_edge[1:]+bin_edge[:-1])/2


/Local/rode/.conda/envs/py36/lib/python3.6/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


In [9]:
res_seas.to_hdf('../c2_kappa_mean_rotation.h5', 'sea_urchins')
np.savetxt("../sea_urchin_spectrum.csv",  np.vstack([freq, power, power_std]).T)

/Local/rode/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Local/rode/.conda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['stuck', 'exp']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
